![](https://user-images.githubusercontent.com/872296/49451747-a04c4500-f7be-11e8-96dc-c8d605249c5e.png)

_Execute code in the selected cell using the Play button at the top (or `ctrl + return`)._

![image](https://user-images.githubusercontent.com/872296/49451632-611df400-f7be-11e8-85a1-5c3a670a0eaf.png)

---

# HackerNews / Algolia Python Library

`python-hn` is a small library to access [Hacker New's Search API](https://hn.algolia.com/api) powered by [Algolia](https://www.algolia.com).

It's designed to be simple to use and pythonic, but also provide the full power of Search API.

## Install instructions

Install with `pip`:

In [99]:
!pip install python-hn

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Usage

The main function of the library is `search_by_date`, which accepts multiple parameters; among them, is a query string containing the phrase to search for.

In [1]:
from hn import search_by_date

In [2]:
results = search_by_date('python')

By default, all the different "post types" will be included: stories, comments, polls, etc.

_This util function will help us format the ouput in a pretty way (content hidden)_

In [3]:
import html
import tabulate

from itertools import islice
from IPython.display import HTML, display


def output_results(results, count=None, url=True, post_type=True, text_length=25):
    if count:
        results = islice(results, count)
    lines = []
    header = ['ID']
    if post_type:
        header += ['Post Type']
    header += ['Text']
    if url:
        header += ['URL']

    for post in results:
        line = [post['objectID']]
        if post_type:
            line += [post['_tags'][0].title()]
        
        text = html.unescape(
            post['title'] or str(post['comment_text'] or '')[:text_length] + '...')
        line += [text]
        if url:
            link = "https://news.ycombinator.com/item?id={}".format(post['objectID'])
            line += [f"<a href={link}>{link}</a>"]
        lines.append(line)
    return display(HTML(tabulate.tabulate(lines, headers=header, tablefmt='html')))

In [4]:
output_results(results, 10)

ID,Post Type,Text,URL
18648191,Story,Python Community Interview with Brian Peterson,https://news.ycombinator.com/item?id=18648191
18648058,Comment,Get a Raspberry Pi for he...,https://news.ycombinator.com/item?id=18648058
18647980,Comment,I have not found my prefe...,https://news.ycombinator.com/item?id=18647980
18647588,Story,How good are your Python skills?,https://news.ycombinator.com/item?id=18647588
18646903,Comment,I already contribute to t...,https://news.ycombinator.com/item?id=18646903
18646545,Comment,DATA ARCHITECT (Location:...,https://news.ycombinator.com/item?id=18646545
18646501,Comment,recursion is your friend ...,https://news.ycombinator.com/item?id=18646501
18646406,Story,Embed Python in NGINX,https://news.ycombinator.com/item?id=18646406
18645690,Story,Show HN: Tracker Radio – Play retro tracker based music in your browser,https://news.ycombinator.com/item?id=18645690
18645293,Comment,"Jashmenn, I just checked ...",https://news.ycombinator.com/item?id=18645293



As you can see, we're receiving both results for stories and comments.

### Searching by author

The second parameter of the `search_by_date` function is the author's username of the post (story or comment). Example:

In [5]:
# 'pg' is Paul Graham: https://news.ycombinator.com/user?id=pg
results = search_by_date('lisp', 'pg')

In [6]:
output_results(results, 10)

ID,Post Type,Text,URL
7446596,Comment,This was all the code it ...,https://news.ycombinator.com/item?id=7446596
7261591,Story,Source of the recent outagelet,https://news.ycombinator.com/item?id=7261591
7128860,Comment,Why doesn't the arti...,https://news.ycombinator.com/item?id=7128860
6705398,Comment,I thought this would be a...,https://news.ycombinator.com/item?id=6705398
6557282,Comment,"Has ""Lisp hacker&quo...",https://news.ycombinator.com/item?id=6557282
6400769,Comment,It's a real person. ...,https://news.ycombinator.com/item?id=6400769
6210178,Comment,Wow is this list good. I...,https://news.ycombinator.com/item?id=6210178
5575255,Story,Danny Sullivan's Twitter list about Watertown,https://news.ycombinator.com/item?id=5575255
5419609,Comment,Unfortunately I can't nam...,https://news.ycombinator.com/item?id=5419609
5376251,Comment,It happens. There are in...,https://news.ycombinator.com/item?id=5376251


### Filtering results by post type

You can pass multiple boolean parameters to filter the post types: `stories`, `comments`, `show_hn`, `ask_hn`, `front_page`, `polls`, `pollopt`.

In [7]:
results = search_by_date('python', stories=True)

In [8]:
output_results(results, 10)

ID,Post Type,Text,URL
18648191,Story,Python Community Interview with Brian Peterson,https://news.ycombinator.com/item?id=18648191
18647588,Story,How good are your Python skills?,https://news.ycombinator.com/item?id=18647588
18646406,Story,Embed Python in NGINX,https://news.ycombinator.com/item?id=18646406
18645690,Story,Show HN: Tracker Radio – Play retro tracker based music in your browser,https://news.ycombinator.com/item?id=18645690
18645552,Story,Ask HN: Is it ethical for companies to commoditize open source projects?,https://news.ycombinator.com/item?id=18645552
18644055,Story,"Vscode IntelliCode: AI-Assisted Productivity for Python, JavaScript, Java",https://news.ycombinator.com/item?id=18644055
18643132,Story,When the allocator works against you: memory fragmentation in Python on glibc,https://news.ycombinator.com/item?id=18643132
18641986,Story,Ask HN: What's the best way to spend $2500 on Machine Learning content?,https://news.ycombinator.com/item?id=18641986
18640499,Story,AWSome: simple python wrapper over boto3 inspired by AWS cli,https://news.ycombinator.com/item?id=18640499
18640457,Story,Python logging made (stupidly) simple,https://news.ycombinator.com/item?id=18640457


Also filtering by author:

In [11]:
results = search_by_date('lisp', author='pg', stories=True)

In [12]:
output_results(results, 10)

ID,Post Type,Text,URL
7261591,Story,Source of the recent outagelet,https://news.ycombinator.com/item?id=7261591
5575255,Story,Danny Sullivan's Twitter list about Watertown,https://news.ycombinator.com/item?id=5575255
5092711,Story,What we discovered about InstallMonetizer,https://news.ycombinator.com/item?id=5092711
2463285,Story,Andreessen Horowitz Leads $1.75M Round In Freebie Marketplace Listia,https://news.ycombinator.com/item?id=2463285
878576,Story,Final Startup School Speaker List (Evan and Biz now coming),https://news.ycombinator.com/item?id=878576
842685,Story,Doug McIlroy: McCarthy Presents Lisp,https://news.ycombinator.com/item?id=842685
93526,Story,"New: Best Comments (via ""Lists"" Link at Bottom)",https://news.ycombinator.com/item?id=93526
35935,Story,Arrington: The FCC Needs To Listen To Google,https://news.ycombinator.com/item?id=35935
7687,Story,"My Pairwise Test Results (not bad, but got the book part wrong)",https://news.ycombinator.com/item?id=7687


Filtering by query, author and including both stories and comments:

In [13]:
results = search_by_date('lisp', author='pg', stories=True, comments=True)

In [14]:
output_results(results, 10)

ID,Post Type,Text,URL
7446596,Comment,This was all the code it ...,https://news.ycombinator.com/item?id=7446596
7261591,Story,Source of the recent outagelet,https://news.ycombinator.com/item?id=7261591
7128860,Comment,Why doesn't the arti...,https://news.ycombinator.com/item?id=7128860
6705398,Comment,I thought this would be a...,https://news.ycombinator.com/item?id=6705398
6557282,Comment,"Has ""Lisp hacker&quo...",https://news.ycombinator.com/item?id=6557282
6400769,Comment,It's a real person. ...,https://news.ycombinator.com/item?id=6400769
6210178,Comment,Wow is this list good. I...,https://news.ycombinator.com/item?id=6210178
5575255,Story,Danny Sullivan's Twitter list about Watertown,https://news.ycombinator.com/item?id=5575255
5419609,Comment,Unfortunately I can't nam...,https://news.ycombinator.com/item?id=5419609
5376251,Comment,It happens. There are in...,https://news.ycombinator.com/item?id=5376251


Other examples:

In [15]:
results = search_by_date('lisp', author='pg', polls=True)

In [16]:
output_results(results, 10)

ID,Post Type,Text,URL
1185896,Poll,Poll: Ask or Leaders in Top Bar?,https://news.ycombinator.com/item?id=1185896


In [17]:
results = search_by_date('python', show_hn=True)

In [18]:
output_results(results, 10)

ID,Post Type,Text,URL
18645690,Story,Show HN: Tracker Radio – Play retro tracker based music in your browser,https://news.ycombinator.com/item?id=18645690
18639121,Story,Show HN: Cog – A Pure Python Graph Database,https://news.ycombinator.com/item?id=18639121
18629029,Story,Show HN: Python 3 Bootcamp,https://news.ycombinator.com/item?id=18629029
18608728,Story,"Show HN: Terracotta, a light-weight XYZ tile server in Python",https://news.ycombinator.com/item?id=18608728
18604364,Story,Show HN: Packagr.app – a cloud hosted PyPI server for Python developers,https://news.ycombinator.com/item?id=18604364
18600380,Story,Show HN: StrictYAML 1.0 released: parse and validate YAML sanely in python,https://news.ycombinator.com/item?id=18600380
18599179,Story,Show HN: Enophp – PHP library for the eno notation language,https://news.ycombinator.com/item?id=18599179
18583789,Story,"Show HN: PowerSong, a platform to see which songs make you run or ride faster",https://news.ycombinator.com/item?id=18583789
18576532,Story,Show HN: A sum-type decorator for python,https://news.ycombinator.com/item?id=18576532
18544818,Story,Show HN: CV Compiler – A Python-based tool to improve tech resumes,https://news.ycombinator.com/item?id=18544818


### Tags

HN Search API supports "tags", that can be logically combined to create complex expressions. There are 3 types of tags included in `python-hn`:

* `PostType`: with options `story`, `comment`, `poll`, `pollopt`, `show_hn`, `ask_hn`, `front_page`.
* `Author`: receives the username as param (`Author('pg')`).
* `StoryID`: receives the story id (`StoryID('6902129')`)

`PostType`s also have aliases that can be used instead of writing the full tag.

In [19]:
from hn import PostType, Author, StoryID
from hn.tags import Story, Comment, AskHN, ShowHN

In [20]:
PostType('story') == Story

True

The power of the tags arise when we combine them logically with `or` and `and` expressions. For example, you could ask for posts that are either _Ask HN_ or _Show HN_. To combine tags with an `or` expression, the operator is `|`:

In [21]:
tags = PostType('ask_hn') | PostType('show_hn')

In [24]:
results = search_by_date('python', tags=tags)

In [25]:
output_results(results, 10)

ID,Post Type,Text,URL
18645690,Story,Show HN: Tracker Radio – Play retro tracker based music in your browser,https://news.ycombinator.com/item?id=18645690
18645552,Story,Ask HN: Is it ethical for companies to commoditize open source projects?,https://news.ycombinator.com/item?id=18645552
18641986,Story,Ask HN: What's the best way to spend $2500 on Machine Learning content?,https://news.ycombinator.com/item?id=18641986
18639121,Story,Show HN: Cog – A Pure Python Graph Database,https://news.ycombinator.com/item?id=18639121
18636437,Story,Ask HN: What's the best and quickest way to build a basic web app for a novice?,https://news.ycombinator.com/item?id=18636437
18629029,Story,Show HN: Python 3 Bootcamp,https://news.ycombinator.com/item?id=18629029
18615990,Story,Ask HN: Does any one use poetry for python dependency management in production?,https://news.ycombinator.com/item?id=18615990
18612013,Story,Ask HN: How to transition from business role to coder,https://news.ycombinator.com/item?id=18612013
18608728,Story,"Show HN: Terracotta, a light-weight XYZ tile server in Python",https://news.ycombinator.com/item?id=18608728
18606914,Story,Ask HN: What programming language to choose for coding interviews?,https://news.ycombinator.com/item?id=18606914


Using aliases:

In [26]:
tags = AskHN | ShowHN

In [27]:
results = search_by_date('python', tags=tags)

In [28]:
output_results(results, 10)

ID,Post Type,Text,URL
18645690,Story,Show HN: Tracker Radio – Play retro tracker based music in your browser,https://news.ycombinator.com/item?id=18645690
18645552,Story,Ask HN: Is it ethical for companies to commoditize open source projects?,https://news.ycombinator.com/item?id=18645552
18641986,Story,Ask HN: What's the best way to spend $2500 on Machine Learning content?,https://news.ycombinator.com/item?id=18641986
18639121,Story,Show HN: Cog – A Pure Python Graph Database,https://news.ycombinator.com/item?id=18639121
18636437,Story,Ask HN: What's the best and quickest way to build a basic web app for a novice?,https://news.ycombinator.com/item?id=18636437
18629029,Story,Show HN: Python 3 Bootcamp,https://news.ycombinator.com/item?id=18629029
18615990,Story,Ask HN: Does any one use poetry for python dependency management in production?,https://news.ycombinator.com/item?id=18615990
18612013,Story,Ask HN: How to transition from business role to coder,https://news.ycombinator.com/item?id=18612013
18608728,Story,"Show HN: Terracotta, a light-weight XYZ tile server in Python",https://news.ycombinator.com/item?id=18608728
18606914,Story,Ask HN: What programming language to choose for coding interviews?,https://news.ycombinator.com/item?id=18606914


To combine tags with an `and` expression, the operator is `&`. For example, all the posts that are **comments** of the story ID [7261591](https://news.ycombinator.com/item?id=7261591):

In [29]:
tags = PostType('comment') & StoryID('7261591')

In [30]:
results = search_by_date(tags=tags)

In [31]:
output_results(results, 10)

ID,Post Type,Text,URL
7298300,Comment,"Sorry, this is uneducated...",https://news.ycombinator.com/item?id=7298300
7269494,Comment,I was able to keep a 10k&...,https://news.ycombinator.com/item?id=7269494
7269489,Comment,"Hmm, that sounds like qui...",https://news.ycombinator.com/item?id=7269489
7268211,Comment,You just need enough core...,https://news.ycombinator.com/item?id=7268211
7266280,Comment,Serving 403s to w00tw00ts...,https://news.ycombinator.com/item?id=7266280
7265942,Comment,"Things like ""Top 10 ...",https://news.ycombinator.com/item?id=7265942
7265025,Comment,Right ... websites with t...,https://news.ycombinator.com/item?id=7265025
7264560,Comment,In this case using nginx ...,https://news.ycombinator.com/item?id=7264560
7262931,Comment,I'm not sure I parti...,https://news.ycombinator.com/item?id=7262931
7262890,Comment,"Interestingly, distinguis...",https://news.ycombinator.com/item?id=7262890


### Filtering by _date_, _points_ or _number of comments_

It is also possible to pass different filters to restrain the search by creation date, number of points or comments. Namely, the parameters are:

* Creation Date: `created_at`
* Points: `points`
* Number of comments: `num_comments`

They accept `>, <, >=, <=` operators with a syntax similar to Django's:

* `lt` (`<`): Lower than. Example `ponts__lt=100`
* `lte` (`<=`): Lower than or equals to. Example `ponts__lte=100`
* `gt` (`>`): Greater than. Example `created_at__gt='2018'` (created after 2018-01-01).
* `gte` (`>=`): Greater than or equals to. Example `num_comments__gte=50`.

A few more examples using filters:

##### All post types created after October 1st, 2018

The filter is `created_at__gt`, `gt` is the `>` operator. The `created_at` filter will try to parse dates automatically (eg: `2018` is interpreted as `2018-01-01`).

In [32]:
results = search_by_date(stories=True, created_at__gt='2018-10')

In [33]:
output_results(results, 10)

ID,Post Type,Text,URL
18648769,Story,Harvard Quietly Amasses California Vineyards – And the Water Underneath,https://news.ycombinator.com/item?id=18648769
18648765,Story,How to stop thinking about code after work,https://news.ycombinator.com/item?id=18648765
18648758,Story,Show HN: Stock Market Forecast Based on Most Similar Historical Patterns,https://news.ycombinator.com/item?id=18648758
18648757,Story,"Jack Dorsey, Posting About Myanmar on Twitter, Is Accused of Being Tone Deaf",https://news.ycombinator.com/item?id=18648757
18648756,Story,Show HN: Failory 2.0 – Learn How to Build a Profitable Startup,https://news.ycombinator.com/item?id=18648756
18648743,Story,Huawei Is China’s Black Mirror Corporation,https://news.ycombinator.com/item?id=18648743
18648742,Story,The Long run developmental effects of Costa Rica’s army abolishment,https://news.ycombinator.com/item?id=18648742
18648733,Story,The Threat of Pseudoscience in India,https://news.ycombinator.com/item?id=18648733
18648727,Story,There's Only One Way to Pay for a Basic Income,https://news.ycombinator.com/item?id=18648727
18648726,Story,Snake your way across your Linux terminal,https://news.ycombinator.com/item?id=18648726


##### Created after October 1st, 2017 and before January 1st 2018

In [34]:
results = search_by_date(stories=True, created_at__gt='2017-10', created_at__lt='2018')

In [35]:
output_results(results, 10)

ID,Post Type,Text,URL
16043719,Story,RAM-less Buffers,https://news.ycombinator.com/item?id=16043719
16043643,Story,Profitable remote job site built with single PHP file (4.5K lines of code),https://news.ycombinator.com/item?id=16043643
16043612,Story,Subreddit Gender Ratios,https://news.ycombinator.com/item?id=16043612
16043585,Story,Core Infrastructure Initiative Best Practices Badge,https://news.ycombinator.com/item?id=16043585
16043578,Story,IOHIDeous OS X Local Kernel Vulnerability,https://news.ycombinator.com/item?id=16043578
16043561,Story,Dave Barry's 2017 Year in Review,https://news.ycombinator.com/item?id=16043561
16043557,Story,New Year’s Resolutions and the Science of Willpower (2015),https://news.ycombinator.com/item?id=16043557
16043554,Story,Top Cryptocurrencies or Altcoins to have in 2018,https://news.ycombinator.com/item?id=16043554
16043548,Story,The Google Book,https://news.ycombinator.com/item?id=16043548
16043542,Story,AI and Deep Learning in 2017 – A Year in Review,https://news.ycombinator.com/item?id=16043542


##### A few more examples:

In [36]:
# Stories with *exactly* 1000 points
results = search_by_date(stories=True, points=1000)

In [37]:
output_results(results, 10)

ID,Post Type,Text,URL
14090063,Story,IndieHackers.com acquired by Stripe,https://news.ycombinator.com/item?id=14090063
13489156,Story,First they came for the Iranians,https://news.ycombinator.com/item?id=13489156


In [38]:
# Stories including "Python" with more than 1000 points
results = search_by_date('python', stories=True, points__gt=1000)

In [39]:
output_results(results, 10)

ID,Post Type,Text,URL
17515492,Story,“I'm basically giving myself a permanent vacation from being BDFL”,https://news.ycombinator.com/item?id=17515492
13319904,Story,Grumpy: Go running Python,https://news.ycombinator.com/item?id=13319904


In [40]:
# Comments with more than 50 points
results = search_by_date('python', comments=True, points__gt=50)

In [41]:
output_results(results, 10)

ID,Post Type,Text,URL
8374713,Comment,"The older I get, the more...",https://news.ycombinator.com/item?id=8374713
8167733,Comment,The idea that Python is s...,https://news.ycombinator.com/item?id=8167733
8070592,Comment,I really think the future...,https://news.ycombinator.com/item?id=8070592
8002089,Comment,I work at Monash Universi...,https://news.ycombinator.com/item?id=8002089
7923883,Comment,So I've finally been...,https://news.ycombinator.com/item?id=7923883
7802095,Comment,That pretty much reiterat...,https://news.ycombinator.com/item?id=7802095
7799923,Comment,I doubt the lazy people w...,https://news.ycombinator.com/item?id=7799923
7733529,Comment,The key to fast startup d...,https://news.ycombinator.com/item?id=7733529
7733148,Comment,Having written a bunch of...,https://news.ycombinator.com/item?id=7733148
7716040,Comment,This is a pretty good lis...,https://news.ycombinator.com/item?id=7716040


In [42]:
# Stories with 100 comments or more
results = search_by_date(stories=True, num_comments__gt=100)

In [43]:
output_results(results, 10)

ID,Post Type,Text,URL
18645269,Story,"Netflix’s biggest competition isn’t sleep, it’s YouTube",https://news.ycombinator.com/item?id=18645269
18644590,Story,LED Strain,https://news.ycombinator.com/item?id=18644590
18644489,Story,Why Not to Use Quora,https://news.ycombinator.com/item?id=18644489
18643427,Story,Why Aren’t Rich People Happy With the Money They Have?,https://news.ycombinator.com/item?id=18643427
18642635,Story,Coinbase is exploring the addition of new currencies,https://news.ycombinator.com/item?id=18642635
18642427,Story,Group led by Thomas Piketty presents plan for ‘a fairer Europe’,https://news.ycombinator.com/item?id=18642427
18642336,Story,JIRA is an antipattern,https://news.ycombinator.com/item?id=18642336
18640880,Story,Show HN: Visalist – Find visa requirements for countries around the world,https://news.ycombinator.com/item?id=18640880
18640755,Story,Crypto Market Crash Leaving Bankrupt Startups in Its Wake,https://news.ycombinator.com/item?id=18640755
18640136,Story,Do I really need to get out the soldering-iron again?,https://news.ycombinator.com/item?id=18640136
